# Housing Prices in University Towns during the Recession

(This analysis is inspired by a MOOC assignment.)

## Introduction
This analysis explores the effect of the Recession on Housing Prices in University Towns.

- The Recession time period is calculated by analyzing the change in the United States GDP quarter over quarter.  
- A list of University Towns is scraped from a Wikipedia page.  
- Housing Prices are analyzed on a quarter-by-quarter basis.  

The hypothesis I am testing is that Housing Prices in University Towns were less affected by the Recession than Housing Prices in non-University Towns.  
I employ a t-test to prove/disprove this hypothesis.

### Definitions  
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.


## Data Sources

1. GDP is pulled from an excel file provided by the Bureau of Economic Analysis.
2. University Towns is scraped from the Wikipedia page (https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) via BeautifulSoup
3. Housing Prices is extracted from a .csv file provided by Zillow. 

### Data Processing

In [187]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import requests
from bs4 import BeautifulSoup


In [188]:
def wikiScrape(url):
    # Return Wikipedia page as string
    website_url = requests.get(url).text
    
    soup = BeautifulSoup(website_url,'lxml')
    return soup.text

In [189]:
def getStates(txt_list):
    # Extract states from list    
    states = pd.Series()
    idx = 0
    for i in range(len(txt_list)):
        if "edit" in txt_list[i]:
            states.at[i] = txt_list[i].replace("[edit]","")

    return states

In [204]:
def getUniversityTowns():
    # Scrape list of University Towns from Wikipedia
    # Return as a dataframe
    
    #Scrape Wikipedia page
    txt = wikiScrape('https://en.wikipedia.org/wiki/List_of_college_towns')
    
    # Extract relevant list of US College Towns
    txt = txt[txt.find('Alabama[edit]'):txt.find('College towns in Argentina[edit]')]
    txt_list = txt.splitlines()

    # Create dataframe
    df = pd.DataFrame({'RegionName': txt_list})
     
    # Get Series of States
    states = getStates(txt_list)
    
    # Add States to dataframe
    df['State'] = states
    df['State'].fillna(method = 'ffill', inplace = True)
    df.drop(states.index, inplace = True)
    
    # Cleanse University Towns
    df['RegionName'] = list(map(lambda x: x.split("(")[0].rstrip(), df['RegionName']))
    df['RegionName'] = list(map(lambda x: x.split("[")[0].rstrip(), df['RegionName']))
    
    return df

preview = getUniversityTowns().head(30)
preview

,RegionName,State
1,Auburn,Alabama
2,Birmingham,Alabama
3,Dothan,Alabama
4,Florence,Alabama
5,Homewood,Alabama
6,Huntsville,Alabama
7,Jacksonville,Alabama
8,Livingston,Alabama
9,Mobile,Alabama
10,Montevallo,Alabama


In [186]:
def getGDP():
    # Extract GDP data from excel file
    gdp = pd.read_excel('gdplev.xls', skiprows = 5)
    
    # Rename and restructure dataframe
    gdp.drop(gdp.index[0:214],inplace = True)
    gdp = gdp[['Unnamed: 4', 'GDP in billions of chained 2009 dollars.1']]
    gdp.columns = ['Quarter','GDP']
    gdp.set_index("Quarter", inplace = True)
    
    return gdp

gdp = readGDP()

In [162]:
def getRecessionStart(gdp):
    # Find year, quarter when Recession began (2 consecutive years of GDP decline)
    
    # Calculate marginal GDP as a dataframe
    gdp_change = gdp.diff()
    
    for i, quarter in enumerate(gdp_change.index):
        if (gdp_change.iloc[i] < 0).bool() and (gdp_change.iloc[i+1] < 0).bool():
            return quarter
    return None  

getRecessionStart(gdp)

'2008q3'

In [163]:
def getRecessionEnd(gdp):
    # Find year, quarter when Recession began 
    # (2 consecutive years of GDP growth after the Recession began)
    
    # Calculate marginal GDP as a dataframe
    gdp_change = gdp.diff()
    
    # Only consider time period after Recession began
    gdp_change = gdp_change.loc[(gdp_change.index >= getRecessionStart(gdp))]
    
    for i, quarter in enumerate(gdp_change.index):
        if (gdp_change.iloc[i-1] > 0).bool() and (gdp_change.iloc[i] > 0).bool():
            return quarter
    return None  

getRecessionEnd(gdp)

'2009q4'

In [164]:
def getRecessionBottom(gdp):
    # Returns year, quarter when GDP was lowest during the Recession
    gdp = gdp.loc[(gdp.index >= getRecessionStart(gdp)) & (gdp.index <= getRecessionEnd(gdp))]
    
    return gdp.loc[gdp['GDP'] == gdp['GDP'].min()].index.item()

getRecessionBottom(gdp)

'2009q2'

In [165]:
def convertToQuarters(col):
    if col.endswith(("01", "02", "03")):
        c = col[:4] + "q1"
    elif col.endswith(("04", "05", "06")):
        c = col[:4] + "q2"
    elif col.endswith(("07", "08", "09")):
        c = col[:4] + "q3"
    else:
        c = col[:4] + "q4"
    return c

In [194]:
def getHousing():
    # Creates dataframe with Housing info
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    
    house = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Structure dataframe; group by State and Town 
    house['State'].replace(states,inplace=True)
    house.set_index(['State','RegionName'],inplace = True)
    house = house.sort_index()
    
    # Remove unneeded columns (i.e. data related to years prior to January, 2000)
    house = house.iloc[:, 49:250]
    
    # Group months into quarters; average housing prices
    house = house.groupby(convertToQuarters, axis = 1).mean()
    
    return house

preview = getHousing().head(15)
preview

2000q1         2000q2         2000q3  \
State   RegionName                                                    
Alabama Adamsville       69033.333333   69166.666667   69800.000000   
        Alabaster       122133.333333  123066.666667  123166.666667   
        Albertville      73966.666667   72600.000000   72833.333333   
        Arab             83766.666667   81566.666667   81333.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81700.000000   78533.333333   79133.333333   
        Bayou La Batre   44066.666667   44500.000000   44266.666667   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       54033.333333   54400.000000   54966.666667   
        Boaz             70866.666667   70266.666667   70300.000000   
        Brent            92933.333333   94333.333333   96166.666667   
        Brighton                  NaN            NaN            NaN   
        Brookwood        92566.666667   95100.000000   98866.666667   

                               2000q4         2001q1         2001q2  \
State   RegionName                                                    
Alabama Adamsville       71966.666667   73466.666667   74000.000000   
        Alabaster       123700.000000  123233.333333  125133.333333   
        Albertville      74200.000000   75900.000000   76000.000000   
        Arab             82966.666667   84200.000000   84533.333333   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      81300.000000   85700.000000   87266.666667   
        Bayou La Batre   43666.666667   42500.000000   43333.333333   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       56066.666667   56833.333333   57600.000000   
        Boaz             71466.666667   72833.333333   71900.000000   
        Brent            98333.333333   96533.333333   98500.000000   
        Brighton                  NaN            NaN            NaN   
        Brookwood        99966.666667  101666.666667  103666.666667   

                               2001q3         2001q4         2002q1  \
State   RegionName                                                    
Alabama Adamsville       73333.333333   73100.000000   73333.333333   
        Alabaster       127766.666667  127200.000000  127300.000000   
        Albertville      72066.666667   73566.666667   76533.333333   
        Arab             81666.666667   83900.000000   87266.666667   
        Ardmore                   NaN            NaN            NaN   
        Axis                      NaN            NaN            NaN   
        Baileyton                 NaN            NaN            NaN   
        Bay Minette      85900.000000   85000.000000   84066.666667   
        Bayou La Batre   45433.333333   45400.000000   45400.000000   
        Bessemer                  NaN            NaN            NaN   
        Birmingham       58433.333333   58700.000000   59500.000000   
        Boaz             68733.333333   69833.333333   72766.666667   
        Brent            99366.666667  104100.000000  103800.000000   
        Brighton                  NaN            NaN            NaN   
        Brookwood       101833.333333   99900.000000   99633.333333   

                               2002q2  ...         2014q2         2014q3  \
State   RegionName                     ...                                 
Alabama Adamsville       73133.333333  ...   77066.666667   75966.666667   
        Alabaster       128000.000000  ...  147133.333333  147633.333333   
        Albertville      76366.666667  ...   84033.333333   84766.666667   
        Arab             87

In [208]:
def t_testing():
    # Get University Towns
    univTowns = getUniversityTowns()
    # Index by State and Town
    univTowns.set_index(['State','RegionName'], inplace = True)

    # Get GDP data|
    gdp = getGDP()
    QBeforeRec = getRecessionStart(gdp)
    QRecBottom = getRecessionBottom(gdp)
    QAfterRec = getRecessionEnd(gdp)
    
    # Get Housing Prices data
    house = getHousing()
    house = house[[QBeforeRec,QRecBottom]].dropna()
    house['ratio'] = house[QBeforeRec]/house[QRecBottom]
    
    
    # Split Housing Prices into University Town and Non-University Town
    univ_house = pd.merge(univTowns, house, how = 'inner', left_index = True, right_index = True)
    non_univ_house = house[~house.index.isin(univ_house.index)]
    ## Verified that there all towns listed in University Towns dataset are included in list of towns in Housing Prices data
    
    # Run statistical analysis
    t_stat, p_value = ttest_ind(univ_house["ratio"], non_univ_house["ratio"])

    if p_value < 0.01:
        different = True
    else:
        different = False
    if t_stat < 0:
        better = "university town"
    else:
        better = "non-university town"
    return (different, p_value, better)

t_testing()

(True, 4.210723869029692e-08, 'non-university town')

## Assumptions
1. Scope of analysis is limited to quarter-by-quarter time periods as that is the deepest level of information provided - limited by GDP
2. I averaged housing prices rather than added them as I am comparing housing prices of individual homes; averaging prices prevents distortion by number of houses in each town

## Conclusion


With a p-value less than .01, we reject the null hypothesis that the Recession affected housing prices equally between University Towns and Non-University Towns.  
Furthermore, the t-stat was positive, indicating price ratio for the University Towns is more than that of the Non-University Towns. In other words, Housing Prices in Non-University Towns were less affected by the recession.

However, since Wikipedia is constantly updated. The list of University Towns pulled directly from the site most likely does not accurately reflect which towns were University Towns during the Recession. Indeed, the number of University Towns pulled during this analysis neared 1000 in number whereas the text file provided by the MOOC listed approximately half that.

## Utilizing an Alternate Data Source

Below, I will utilize the list provided by the MOOC to perform my analysis to investigate the potential difference.

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    text_file = open("university_towns.txt")

    states = pd.Series({idx: lines.strip().replace("[edit]", "")
             for idx,lines in enumerate(text_file) if "edit" in lines})
    
    univ_town = pd.read_csv('university_towns.txt', sep = '\n', header = None)
    univ_town.rename(columns = {univ_town.columns[0] : 'RegionName'}, inplace = True)

    
    univ_town['State'] = states

    univ_town=univ_town.fillna(method='ffill')
    univ_town=univ_town.drop(states.index)
    
    univ_town['RegionName'] = list(map(lambda x: x.split("(")[0].rstrip(),univ_town['RegionName']))
    
    #univ_town.set_index(['State', 'RegionName'], inplace = True)
    
    return univ_town

get_list_of_university_towns()

In [210]:
def t_testing():
    # Get University Towns
    univTowns = get_list_of_university_towns()
    # Index by State and Town
    univTowns.set_index(['State','RegionName'], inplace = True)

    # Get GDP data|
    gdp = getGDP()
    QBeforeRec = getRecessionStart(gdp)
    QRecBottom = getRecessionBottom(gdp)
    QAfterRec = getRecessionEnd(gdp)
    
    # Get Housing Prices data
    house = getHousing()
    house = house[[QBeforeRec,QRecBottom]].dropna()
    house['ratio'] = house[QBeforeRec]/house[QRecBottom]
    
    
    # Split Housing Prices into University Town and Non-University Town
    univ_house = pd.merge(univTowns, house, how = 'inner', left_index = True, right_index = True)
    non_univ_house = house[~house.index.isin(univ_house.index)]
    ## Verified that there all towns listed in University Towns dataset are included in list of towns in Housing Prices data
    
    # Run statistical analysis
    t_stat, p_value = ttest_ind(univ_house["ratio"], non_univ_house["ratio"])

    if p_value < 0.01:
        different = True
    else:
        different = False
    if t_stat < 0:
        better = "university town"
    else:
        better = "non-university town"
    return (different, p_value, better)

t_testing()

(True, 0.005496427353694603, 'university town')

# Follow-up

Confirming my assumption, the the results changed significantly - altering the decision result. Now, with a smaller list of University Towns, we see that the t-stat is become negative, indicating that Housing Prices in University Towns were less affected by the Recession.